In [1]:
import sys
from pathlib import Path

current_dir = Path('.')
current_dir = current_dir.absolute()
root_dir = current_dir.parent.parent
src_dir = root_dir / 'src'

sys.path.append(str(src_dir))

%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv(str(src_dir / '.env'))

True

In [2]:
from api.containers import analyze_container

analyze = analyze_container.service()
df = analyze.collect_all()
df

F:\Projects\hltv_notebook\src\infra\data_frames\pipeline\matches.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[C.Winner] = winners
F:\Projects\hltv_notebook\src\infra\data_frames\pipeline\matches.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  part_df.rename(columns=dict(zip(part, new_columns)), inplace=True)
F:\Projects\hltv_notebook\src\infra\data_frames\pipeline\matches.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

,Team,Score,Map,Opponent,Opponent Score,Date,Event,Is Winner
0,Budapest Five,5,inf Inferno,Katuna,16,26/2/23,CCT South Europe Series 3,False
1,Katuna,14,anc Ancient,Budapest Five,16,26/2/23,CCT South Europe Series 3,False
2,Budapest Five,12,ovp Overpass,Katuna,16,26/2/23,CCT South Europe Series 3,False
3,ECSTATIC,14,inf Inferno,Astralis Talent,16,26/2/23,CCT North Europe Series 4 Closed Qualifier,False
4,Eternal Fire,15,ovp Overpass,fnatic,19,26/2/23,ESL Pro League Season 17,False
...,...,...,...,...,...,...,...,...
50715,Virtus.pro,7,ovp Overpass,Spirit,16,29/6/21,StarLadder CIS RMR 2021,False
50716,Virtus.pro,13,d2 Dust2,Spirit,16,29/6/21,StarLadder CIS RMR 2021,False
50717,Virtus.pro,16,inf Inferno,Spirit,11,29/6/21,StarLadder CIS RMR 2021,True
50718,Gambit,16,d2 Dust2,FORZE,6,29/6/21,StarLadder CIS RMR 2021,True


In [ ]:
# Dump into CSV file

prefix = datetime.utcnow().strftime('%d%m%Y_%H%M%S')

teams.to_csv(f'../data/{prefix}_teams.csv')

In [ ]:
sum_teams = teams.groupby('Team').sum('Score')
sum_teams

In [ ]:
sum_teams['Matches'] = teams.groupby('Team').size()
sum_teams

In [ ]:
sum_teams['Winrate'] = sum_teams['Is Winner'] * 100 / sum_teams['Matches']
sum_teams.sort_values('Matches', ascending=False)

In [ ]:
sum_teams[sum_teams['Matches']> 15].sort_values('Winrate', ascending=False)